In [542]:
%run common.py

Planning start time: Jul 22, 14:40 (1469187638)


In [543]:
'''
    Examples:
    al = pd.read_csv(FOLDER + '/mandatory/travel_times_all_pairs.csv', sep=';')
    get_longest_pair(['МАРИИНСК', 'ИЛАНСКАЯ', 'ИРКУТСК-СОРТИРОВОЧНЫЙ', 'РЕШОТЫ'], al.set_index(['st_from_name', 'st_to_name']))
    
    => Out[460]: ('МАРИИНСК', 'ИРКУТСК-СОРТИРОВОЧНЫЙ', 94170)
    
    get_longest_pair(['МАРИИНСК', 'ИЛАНСКАЯ', 'РЕШОТЫ', 'ИРКУТСК-СОРТИРОВОЧНЫЙ'], nx.all_pairs_dijkstra_path_length(g))
    
    => Out[460]: ('МАРИИНСК', 'ИРКУТСК-СОРТИРОВОЧНЫЙ', 94170)    
'''

def get_longest_pair(st_list, lengths):
    sm1, sm2 = '', ''
    m = 0
    for s1 in st_list:
        for s2 in st_list:
            if type(lengths) == dict:
                l = lengths[s1][s2]
            elif type(lengths) == pd.DataFrame:
                l = lengths.ix[s1, s2].values[0]
            else: l = 0
            if l > m:
                m = l
                sm1, sm2 = s1, s2
    return (sm1, sm2, m)

In [544]:
all_lengths = pd.read_csv(FOLDER + '/mandatory/travel_times_all_pairs.csv', sep=';').set_index(['st_from_name', 'st_to_name'])
    
def get_reg_name(l):
    l_big = [st for st in l if st in big_st]
    if len(l_big) == 2:
        ret = l_big
    elif len(l_big) > 2:
        st1, st2, length = get_longest_pair(l_big, all_lengths)
        ret = [st1, st2]
    else:
        st1, st2, length = get_longest_pair(l, all_lengths)
        ret = [st1, st2]
    return ret[0] + ' - ' + ret[1]        

team_region = pd.read_csv(FOLDER + 'team_region.csv', dtype={'st_from':str, 'st_to':str, 'depot':str})
add_info(team_region)
big_st = stations[stations.norm_time > 0].name.unique()
team_region['depot_name'] = team_region.depot.map(st_names.name)
team_region['reg_name'] = team_region.team_region\
                            .map(team_region.groupby('team_region').st_from_name.unique().apply(get_reg_name))

cols_tracks = ['team_region', 'asoup', 'depot', 'depot_name', 'st_from_name', 'st_to_name', 'reg_name']
cols_times = ['team_region', 'asoup', 'depot', 'depot_name', 'time_f', 'time_b', 'time_wr']

In [545]:
#print(nice_time(current_time + 20*60 + 3*3600))
nct = current_time + 20*60 + 3*3600
st1 = 'ИРКУТСК-СОРТИРОВОЧНЫЙ'
start_st = team_plan[team_plan.state.isin([0, 1])].drop_duplicates('team').set_index('team')
team_plan['team_type'] = team_plan.team.apply(lambda x: int(str(x)[0]))
team_plan['start_st'] = team_plan.team.map(start_st.st_from)
team_plan.depot.fillna(team_plan.start_st, inplace=True)
team_plan['depot_name'] = team_plan.depot.map(st_names.name)
cols = ['team', 'st_from_name', 'st_to_name', 'time_start_norm', 'time_end_norm', 'state', 'loco']
a = team_plan[(team_plan.time_start >= nct) & (team_plan.time_start < nct + 24 * 3600)
         & (team_plan.state.isin([0, 1])) & (team_plan.st_from_name == st1) & (team_plan.depot_name == st1)]
print(a.st_to_name.value_counts())
print(a.state.value_counts())
print(a.team_type.value_counts())

ГОНЧАРОВО          24
БАТАРЕЙНАЯ         24
ВОЕННЫЙ ГОРОДОК     2
Name: st_to_name, dtype: int64
1    50
Name: state, dtype: int64
2    43
7     7
Name: team_type, dtype: int64


In [546]:
log.head()

,state,text,dt
0,DEBUG,ntf ping stat for: 10.160.22.217:33331 since:...,2016-07-17 00:00:03
1,DEBUG,Изменена задача mnemoscheme.calculate_windows ...,2016-07-17 00:00:09
2,INFO,-----> Ntf size stats:,2016-07-17 00:00:38
3,INFO,"----> Ntf handle time stat {""PropertyNotifier|...",2016-07-17 00:00:38
4,DEBUG,ntf ping stat for: 10.160.22.217:33331 since:...,2016-07-17 00:01:03


In [547]:
print(nice_time(current_time))
cols = ['team', 'st_from_name', 'st_to_name', 'oper_time_f', 'time_start_norm', 'state', 'wait_ct', 'wait']
team_plan['sinfo'] = team_plan['state_info']
team_plan['oper_time_f'] = team_plan.oper_time.apply(nice_time)
team_plan['wait_ct'] = np.round((current_time - team_plan.oper_time) / 3600, 2)
team_plan['wait'] = np.round((team_plan.time_start - team_plan.oper_time) / 3600, 2)
a = team_plan[(team_plan.state_info == '2') 
          & (team_plan.state.isin([0, 1]))][cols].drop_duplicates('team').sort_values('wait', ascending=False)

(a.wait - a.wait_ct).describe()

Jul 22, 14:40


count    76.000000
mean      3.386579
std       2.259012
min       0.000000
25%       1.410000
50%       3.320000
75%       5.320000
max       9.740000
dtype: float64

In [548]:
st = 'ИРКУТСК-СОРТИРОВОЧНЫЙ'

# trains
train_plan['train_type'] = train_plan.train.apply(lambda x: int(str(x)[0]))
trains = train_plan[(train_plan.time_start >= current_time) & (train_plan.time_start < current_time + 24 * 3600)
          & (train_plan.train_type.isin([2, 9]))
          & (train_plan.st_from_name == st)]

print('Trains:')
print(trains.st_to_name.value_counts().to_string(), '\n-')
print(trains.train_type.value_counts().to_string())

locos = loco_plan[(loco_plan.time_start >= current_time) & (loco_plan.time_start < current_time + 24 * 3600)          
          & (loco_plan.st_from_name == st)]
print('\nLocos:')
print(locos[locos.state == 1].st_to_name.value_counts().to_string(), '\n-')
print(locos.state.value_counts().to_string())

team_plan['team_type'] = team_plan.team.apply(lambda x: int(str(x)[0]))
team_plan['start_st'] = team_plan.team.map\
            (team_plan[team_plan.state.isin([0, 1])].drop_duplicates('team').set_index('team').st_from)
team_plan.depot.fillna(team_plan.start_st, inplace=True)
team_plan['depot_name'] = team_plan.depot.map(st_names.name)
teams = team_plan[(team_plan.time_start >= current_time) & (team_plan.time_start < current_time + 24 * 3600)          
          & (team_plan.st_from_name == st)]
print('\nTeams:')
print(teams[teams.state == 1].st_to_name.value_counts().to_string(), '\n-')
print(teams[teams.state == 1].team_type.value_counts().to_string(), '\n-')
print(teams[teams.state.isin([0, 1])].state.value_counts().to_string(), '\n-')
print(teams[teams.state.isin([0, 1])].depot_name.value_counts().to_string())

Trains:
ГОНЧАРОВО          59
БАТАРЕЙНАЯ         40
ВОЕННЫЙ ГОРОДОК     2 
-
2    80
9    21

Locos:
ГОНЧАРОВО          46
БАТАРЕЙНАЯ         40
ВОЕННЫЙ ГОРОДОК     2 
-
1    88
0    45
4     6

Teams:
БАТАРЕЙНАЯ         77
ГОНЧАРОВО          52
ВОЕННЫЙ ГОРОДОК     2 
-
2    110
7     21 
-
1    131
0      6 
-
ЗИМА                     58
ИРКУТСК-СОРТИРОВОЧНЫЙ    53
НИЖНЕУДИНСК               5
СЛЮДЯНКА I                5
УЛАН-УДЭ                  3
ГОНЧАРОВО                 2
АНГАСОЛКА                 2
СЛЮДЯНКА II               2
КАЯ                       2
КИТОЙ                     1
КАСЬЯНОВКА                1
МЫСОВАЯ                   1
БАЙКАЛЬСК                 1
КИЖА                      1


In [549]:
all_lengths['dist'] = np.round(all_lengths.tt * 45 / 3600)

In [550]:
routes = pd.read_csv(FOLDER + 'routes.csv', dtype={'st_from':str, 'st_to':str, 'train':str})
add_info(routes)
routes['first_st'] = routes.st_from_name
routes['end_st'] = routes.train.map(routes.drop_duplicates('train', keep='last').set_index('train').st_to_name)
rs = routes.drop_duplicates('train').copy(deep=True)
rs['route'] = rs.apply(lambda row: (row.first_st, row.end_st), axis=1)
rs['tt'] = rs.route.map(lambda x: all_lengths.ix[x[0], x[1]].tt)
rs['dist'] = rs.route.map(lambda x: all_lengths.ix[x[0], x[1]].dist)

In [551]:
print(nice_time(current_time))
train_plan['train_time'] = list(zip(train_plan.train, train_plan.time_start))
loco_plan['train_time'] = list(zip(loco_plan.train, loco_plan.time_start))
train_plan['loco'] = train_plan.train_time.map(loco_plan.drop_duplicates('train_time').set_index('train_time').loco)
train_plan['route'] = train_plan.train.map(rs.set_index('train').route)
train_plan['tt'] = train_plan.train.map(rs.set_index('train').tt)
train_plan['dist'] = train_plan.train.map(rs.set_index('train').dist)
cols = ['train','weight', 'st_from_name', 'st_to_name', 'time_start_norm', 'loco', 'route', 'tt', 'dist']
no_loco = train_plan[(train_plan.time_start < current_time + 24 * 3600) & (train_plan.loco.isnull())]
no_loco.drop_duplicates('train').st_from_name.value_counts().head(10)

Jul 22, 14:40


БИРА                                  84
ВОЛОЧАЕВКА I                          55
ТАЙШЕТ                                19
ТЫНДА                                 16
КОМСОМОЛЬСК-НА-АМУРЕ-СОРТИРОВОЧНЫЙ    14
ФЕВРАЛЬСК                             14
ТАКСИМО                               13
МАРИИНСК                              13
ЗАБАЙКАЛЬСК                           12
ОБЛУЧЬЕ                               11
Name: st_from_name, dtype: int64

In [552]:
st_name = 'КАРЫМСКАЯ'
tr = no_loco.drop_duplicates('train')
cols = ['train','weight', 'st_from_name', 'st_to_name', 'time_start_norm', 'loco', 'route']
print(tr[tr.st_from_name == st_name].sort_values('time_start')[cols].to_string(index=False))

        train  weight st_from_name st_to_name time_start_norm loco                route
 200021359367       0    КАРЫМСКАЯ     ЧИТА I   Jul 22, 07:05  NaN  (КАРЫМСКАЯ, ЧИТА I)


In [553]:
loco_cols = ['loco', 'regions', 'ser_name', 'sections', 'loc_name', 'oper_time_f', 'first_train', 'tts', 'dts']
loco_info['ser_name'] = loco_info.series.map(loco_series.set_index('ser_id').ser_name)
loco_info['oper_time_f'] = loco_info.oper_time.map(nice_time)
loco_starts = loco_plan.drop_duplicates('loco').set_index('loco')
loco_info['first_train'] = loco_info.loco.map(loco_starts.train)
with pd.option_context('display.max_colwidth', 30):
    print(loco_info[loco_info.loc_name == st_name][loco_cols].to_string(index=False))

         loco                        regions ser_name  sections   loc_name    oper_time_f     first_train     tts     dts
 200200083577                 ['2002119299']    ВЛ80Р         3  КАРЫМСКАЯ  Jul 22, 13:51  88880013664307  342000     920
 200200090201   ['2002119294', '2002119320']    2ЭС5К         4  КАРЫМСКАЯ  Jul 22, 13:46  88880013444888   90000  999999
 200200105151                 ['2002119294']    3ЭС5К         3  КАРЫМСКАЯ  Jul 22, 07:59    200020753480  432000    4000
 200200103448   ['2002119294', '2002119299']    3ЭС5К         3  КАРЫМСКАЯ  Jul 22, 14:00  88880013344415  424800  999999
 200200084153                 ['2002119299']    ВЛ80Р         3  КАРЫМСКАЯ  Jul 22, 01:11    230200034040  327600    3138
 200200094891   ['2002119294', '2002119299']    3ЭС5К         3  КАРЫМСКАЯ  Jul 22, 14:06  88880002164575   64800  999999
 200200046737                 ['2002119288']   2ТЭ10М         2  КАРЫМСКАЯ  Jul 22, 10:16  88880005614792  273600  999999
 200200079286   ['200211

In [554]:
loco_plan['ser_name'] = loco_plan.series.map(loco_series.set_index('ser_id').ser_name)
loco_plan['loco_end'] = loco_plan.loco != loco_plan.loco.shift(-1)
loco_plan['next_train'] = loco_plan.train.shift(-1)
loco_plan.loc[loco_plan.loco_end == True, 'next_train'] = -1
loco_ends = loco_plan.drop_duplicates(subset=['loco', 'train'], keep='last')
loco_cols = ['loco', 'regions', 'ltype', 'ser_name', 'sections', 'train', 'time_end_norm', 'next_train', 'tts', 'dts']
loco_ends[(loco_ends.st_to_name == st_name)
         & (loco_ends.time_end < current_time + 24 * 3600)
         & (loco_ends.ltype == 1)
         & (loco_ends.next_train == -1)].sort_values('time_end')[loco_cols]

,loco,regions,ltype,ser_name,sections,train,time_end_norm,next_train,tts,dts
55134,200200106181,"['2002119301', '2002119299']",1,2ЭС5К,4,220209867626,"Jul 22, 09:47",-1,219600,2056
50018,200200081388,['2002119294'],1,3ЭС5К,3,200020338131,"Jul 22, 16:40",-1,162000,255
8974,200200097727,['2002119299'],1,ВЛ80Р,3,220211846119,"Jul 22, 17:34",-1,999999,51
4318,200200089658,['2002119294'],1,3ЭС5К,3,220211632278,"Jul 22, 18:41",-1,999999,791
6038,200200047221,['2002119294'],1,3ЭС5К,3,220211755551,"Jul 22, 19:29",-1,90000,2019
38170,200200084068,"['2002119301', '2002119299']",1,2ЭС5К,4,230200205473,"Jul 22, 21:07",-1,158400,903
29406,200200107673,['2002119294'],1,3ЭС5К,3,220212065227,"Jul 23, 00:26",-1,172800,1495
22404,200200139217,['2002119294'],1,3ЭС5К,3,200020354565,"Jul 23, 00:47",-1,180000,1496
13947,200200096030,['2002119294'],1,3ЭС5К,3,200020481680,"Jul 23, 01:07",-1,194400,46602
19872,200200095906,['2002119294'],1,3ЭС5К,3,220211831268,"Jul 23, 02:30",-1,144000,18438


In [555]:
loco_cols = ['loco', 'st_from_name', 'st_to_name', 'time_start_norm', 'time_end_norm', 'train']
loco_plan[loco_plan.loco == '200200094024'][loco_cols]

,loco,st_from_name,st_to_name,time_start_norm,time_end_norm,train
49027,200200094024,ЧЕРНОРЕЧЕНСКАЯ,БУГАЧ,"Jul 22, 12:30","Jul 22, 15:17",200020959409
49028,200200094024,БУГАЧ,КРАСНОЯРСК-СЕВЕРНЫЙ,"Jul 22, 15:17","Jul 22, 15:39",200020959409
49029,200200094024,КРАСНОЯРСК-СЕВЕРНЫЙ,КРАСНОЯРСК-ВОСТОЧНЫЙ,"Jul 22, 15:39","Jul 22, 15:50",200020959409
49030,200200094024,КРАСНОЯРСК-ВОСТОЧНЫЙ,ЗЫКОВО,"Jul 22, 18:15","Jul 22, 18:24",200020959409
49031,200200094024,ЗЫКОВО,УЯР,"Jul 22, 18:24","Jul 22, 20:18",200020959409
49032,200200094024,УЯР,ЗАОЗЕРНАЯ,"Jul 22, 20:18","Jul 22, 20:55",200020959409
49033,200200094024,ЗАОЗЕРНАЯ,ИЛАНСКАЯ,"Jul 22, 20:55","Jul 22, 22:48",200020959409
49034,200200094024,ИЛАНСКАЯ,РЕШОТЫ,"Jul 22, 22:48","Jul 23, 00:09",200020959409
49035,200200094024,РЕШОТЫ,КЛЮЧИ,"Jul 23, 00:09","Jul 23, 00:26",200020959409
49036,200200094024,КЛЮЧИ,ЮРТЫ,"Jul 23, 00:26","Jul 23, 00:50",200020959409


In [556]:
loco_cols = ['loco', 'st_from_name', 'st_to_name', 'time_start_norm', 'train', 'all_trains']
loco_plan['all_trains'] = loco_plan.loco.map(loco_plan.groupby('loco').train.unique())
a = loco_plan.drop_duplicates(['loco', 'train'])
b = a[(a.time_start < current_time + 24 * 3600) 
    & (a.state == 0) & (a.st_from_name == st_name)][loco_cols]

In [557]:
def is_useful(train, all_trains):
    if train == all_trains[-1]:
        return False
    else:
        i = list(all_trains).index(train)
        next_tr = all_trains[i + 1]
        if next_tr[0] in ['2', '9']:
            return True
        else:
            return False

#b['is_last'] = b[['train', 'all_trains']].apply(lambda row: row.all_trains[-1] == row.train, axis=1)
b['is_useful'] = b[['train', 'all_trains']].apply(lambda row: is_useful(row.train, row.all_trains), axis=1)
b[b.is_useful == False].st_to_name.value_counts()

ЧИТА I     25
ТАРСКАЯ     4
Name: st_to_name, dtype: int64

In [558]:
routes['link_name'] = list(zip(routes.st_from_name, routes.st_to_name))
a = routes[routes.train == '200020313247'].link_name.unique()

In [559]:
loco_tonnage = pd.read_csv(FOLDER + 'loco_tonnage.csv', dtype={'st_from':str, 'st_to':str})
add_info(loco_tonnage)
loco_tonnage['link_name'] = list(zip(loco_tonnage.st_from_name, loco_tonnage.st_to_name))
b = loco_tonnage[loco_tonnage.link_name.isin(a)]

In [560]:
loco_tonnage['ser_name'] = loco_tonnage.series.map(loco_series.set_index('ser_id').ser_name)
loco_tonnage.dropna(subset=['st_from_name', 'st_to_name'], how='any', inplace=True)
b = loco_tonnage.groupby(['link_name', 'ser_name', 'sections']).max_weight.unique().reset_index()
b['ln_cat'] = pd.Categorical(b.link_name, categories=a, ordered=True)
c = b[(b.ln_cat.notnull()) & b.ser_name.apply(lambda x: ('ВЛ8' in x) | ('ЭС5К' in x))].sort_values('ln_cat')

In [561]:
c['very_max'] = c.max_weight.apply(lambda x: x.max())
c[c.very_max > 6000].sort_values(['ln_cat', 'very_max'])

,link_name,ser_name,sections,max_weight,ln_cat,very_max


1. Тайшет-Зима-Китой: 6300 только для ВЛ80Р
2. Большой Луг-Андриановская, Андриановская-Ангасолка: заданы нормы только для тепловозов и ВЛ60.

In [562]:
cols = ['link_name', 'ser_name', 'sections', 'max_weight']
loco_tonnage[(loco_tonnage.st_from_name == 'Андриановская'.upper()) & (loco_tonnage.st_to_name == 'Ангасолка'.upper())]\
    .sort_values('ser_name')[cols]

,link_name,ser_name,sections,max_weight
4757,"(АНДРИАНОВСКАЯ, АНГАСОЛКА)",2ТЭ10В,2,1800
4756,"(АНДРИАНОВСКАЯ, АНГАСОЛКА)",2ТЭ10ВК,2,1800
4763,"(АНДРИАНОВСКАЯ, АНГАСОЛКА)",2ТЭ10Л,2,1800
4762,"(АНДРИАНОВСКАЯ, АНГАСОЛКА)",2ТЭ10М,2,1800
4764,"(АНДРИАНОВСКАЯ, АНГАСОЛКА)",2ТЭ10МК,2,1800
4777,"(АНДРИАНОВСКАЯ, АНГАСОЛКА)",2ТЭ10С,2,1800
4770,"(АНДРИАНОВСКАЯ, АНГАСОЛКА)",2ТЭ10Т,1,1800
4771,"(АНДРИАНОВСКАЯ, АНГАСОЛКА)",2ТЭ10У,2,1800
4772,"(АНДРИАНОВСКАЯ, АНГАСОЛКА)",2ТЭ10УК,2,1800
4773,"(АНДРИАНОВСКАЯ, АНГАСОЛКА)",2ТЭ10УТ,2,1800


In [563]:
prst = pd.read_csv(FOLDER + 'mandatory/priority_team_change_stations.csv', sep=';')
prst['nearest'] = prst.name.apply(lambda x: all_lengths[all_lengths > 0].ix[x].tt.min())
prst.nearest.mean()

3160.690909090909

In [564]:
team_region = pd.read_csv(FOLDER + 'team_region.csv')
team_region.time_f.mean() / 3600 - 2

nan

### Поезда, которые по номеру являются локомотивами резервом, но индекс начинается не с 0001

In [565]:
train_info['oper_time_f'] = train_info.oper_time.apply(nice_time)
train_info['ind_start'] = train_info.ind434.apply(lambda x: x[:4])
train_info['loco'] = train_info.train.map(loco_info.groupby('train').loco.unique())
train_info['lstate'] = train_info.train.map(loco_info.groupby('train').state.unique())
train_info['lstate'] = train_info.lstate.apply(lambda x: x if np.isnan(x) else x[0])
train_info['loco'] = train_info.loco.apply(lambda x: x if type(x) == float else (x[0] if len(x) == 1 else x))
train_info['in_plan'] = train_info.train.isin(train_plan.train)
train_info['loco_p'] = train_info.train.map(loco_plan.sort_values('time_start').drop_duplicates('train').set_index('train').loco)

In [566]:
cols = ['train', 'number', 'ind434', 'loc_name', 'loco', 'ltype', 'lstate', 'loco_p']
train_res = train_info[(train_info.number >= 4000) 
                       & (train_info.number < 5000)]
train_res['ltype'] = train_res.loco.map(loco_info.set_index('loco').ltype)
#nice_print(train_res[train_res.ind_start != '0001'].sort_values('number')[cols], num=True)
#nice_print(train_res[train_res.in_plan == False][cols], num=True)
nice_print(train_res[train_res.loco_p.isnull()].sort_values('number')[cols], num=True)
#nice_print(train_res[cols], num=True)

           train  number         ind434                                    loc_name          loco  ltype  lstate loco_p
0   200200717842    4202  0001-572-2387                      НОВАЯ ЧАРА - ИКАБЬЯКАН  200200103583      1       1    NaN
1   200021410010    4301  9043-877-9044                        НОВАЯ ЧАРА - СЮЛЬБАН  200200051501      0       1    NaN
2   200021413699    4302  0001-663-9369                       ЗАУДИНСКИЙ - ЗАГУСТАЙ  200200062130      0       1    NaN
3   200021437598    4304  0001-005-9423                          ЧИТА I - КАРЫМСКАЯ  200200053446      0       1    NaN
4   200021317546    4401  0001-937-9700                       БИКИН - РОЗЕНГАРТОВКА  200200075015      1       1    NaN
5   200021350636    4402  0001-601-9171                     ФЕВРАЛЬСК - НОВЫЙ УРГАЛ  200200041865      0       1    NaN
6   200021351279    4403  9875-929-9883                        ГВОЗДЕВО - СУХАНОВКА  200200053764      0       1    NaN
7   200021437327    4405  0001-689-9047 

C:\Users\oracle\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [567]:
#pd.set_option('display.width', 500)
print(train_res[cols].ltype.value_counts())
train_res[(train_res.ltype == 1) & (train_res.loco == train_res.loco_p)][cols].head()

0    71
1    59
Name: ltype, dtype: int64


,train,number,ind434,loc_name,loco,ltype,lstate,loco_p
37,200021465840,4041,0001-118-9330,СЛЮДЯНКА I - СЛЮДЯНКА II,200200032959,1,1,200200032959
118,200021427967,4029,0001-230-9330,АНДРИАНОВСКАЯ - БОЛЬШОЙ ЛУГ,200200092725,1,1,200200092725
183,200021436029,4402,0001-258-9047,ХАНИ - ОЛЕКМА,200200097942,1,1,200200097942
184,200021447604,4033,0001-893-9330,АНДРИАНОВСКАЯ - БОЛЬШОЙ ЛУГ,200200103181,1,1,200200103181
244,200021407098,4403,9769-043-9737,СПАССК-ДАЛЬНИЙ - ШМАКОВКА,200200101092,1,1,200200101092


In [568]:
cols = ['loco', 'ltype', 'oper_time_f', 'loc_name', 'train', 'train_number', 'state']
loco_info['train_number'] = loco_info.train.map(train_info.set_index('train').number)
nice_print(loco_info[(loco_info.ltype == 0) 
                      & (loco_info.train == '-1')                     
                      & (loco_info.st_from == '-1')].sort_values('train_number')[cols])

         loco  ltype    oper_time_f                            loc_name train  train_number  state
 200200095345      0  Jul 22, 12:20                              ЧУНОЯР    -1           NaN     -1
 200200073016      0  Jul 22, 12:59                      СПАССК-ДАЛЬНИЙ    -1           NaN     -1
 200200028694      0  Jul 21, 21:58                         ИЗВЕСТКОВАЯ    -1           NaN     -1
 200200096339      0  Jul 22, 03:22                        ХАБАРОВСК II    -1           NaN     -1
 200200088604      0  Jul 22, 13:18                                 NaN    -1           NaN     -1
 200200063550      0  Jul 22, 12:40                           СИБИРЦЕВО    -1           NaN     -1
 200200043190      0  Jul 22, 13:41                     ЕРОФЕЙ ПАВЛОВИЧ    -1           NaN     -1
 200200091223      0  Jul 22, 14:37                              ТАЙШЕТ    -1           NaN     -1
 200200069558      0  Jul 22, 14:00                              ДЗЕМГИ    -1           NaN     -1
 200200070

In [569]:
slot = pd.read_csv(FOLDER + 'slot.csv', dtype={'st_from':str, 'st_to':str})
add_info(slot)
slot_routes = slot.drop_duplicates('slot')[['slot', 'st_from_name', 'time_start']].set_index('slot')\
        .join(slot.drop_duplicates('slot', keep='last')[['slot', 'st_to_name', 'time_end']].set_index('slot'))
slot_routes = slot_routes[['st_from_name', 'st_to_name', 'time_start', 'time_end']]

In [570]:
ns = [9205003038, 9205004359]
team_info['oper_time_f'] = team_info.oper_time.apply(nice_time)
print(team_info[team_info.number.isin(ns)][['team', 'number', 'ttype', 'oper_time_f', 'loc_name', 'state']])

Empty DataFrame
Columns: [team, number, ttype, oper_time_f, loc_name, state]
Index: []


In [571]:
loco_info[loco_info.loco == '200200082272'][['loco', 'number', 'ser_name', 'regions', 'loc_name', 'oper_time_f', 'state', 'train', 'ltype']]

,loco,number,ser_name,regions,loc_name,oper_time_f,state,train,ltype
1240,200200082272,24,ВЛ85,['2002119299'],ИРКУТСК-СОРТИРОВОЧНЫЙ,"Jul 22, 09:36",-1,-1,1


In [572]:
loco_plan[loco_plan.loco == '200200082272'][['loco', 'st_from_name', 'st_to_name', 'time_start_norm', 'time_end_norm', 'train']]

,loco,st_from_name,st_to_name,time_start_norm,time_end_norm,train
1206,200200082272,ИРКУТСК-СОРТИРОВОЧНЫЙ,БАТАРЕЙНАЯ,"Jul 22, 20:32","Jul 22, 20:34",88880003124255
1207,200200082272,БАТАРЕЙНАЯ,МЕГЕТ,"Jul 22, 20:34","Jul 22, 20:42",88880003124255
1208,200200082272,МЕГЕТ,СУХОВСКАЯ,"Jul 22, 20:42","Jul 22, 20:54",88880003124255
1209,200200082272,СУХОВСКАЯ,АНГАРСК,"Jul 22, 20:54","Jul 22, 20:59",88880003124255
1210,200200082272,АНГАРСК,КИТОЙ,"Jul 22, 20:59","Jul 22, 21:08",88880003124255
1211,200200082272,КИТОЙ,МАЛЬТА,"Jul 22, 21:08","Jul 22, 21:50",88880003124255
1212,200200082272,МАЛЬТА,КАСЬЯНОВКА,"Jul 22, 21:50","Jul 22, 22:43",88880003124255
1213,200200082272,КАСЬЯНОВКА,ЧЕРЕМХОВО,"Jul 22, 22:43","Jul 22, 23:03",88880003124255
1214,200200082272,ЧЕРЕМХОВО,ЗАЛАРИ,"Jul 22, 23:03","Jul 23, 00:16",88880003124255
1215,200200082272,ЗАЛАРИ,ЗИМА,"Jul 23, 00:16","Jul 23, 01:12",88880003124255


In [579]:
d = datetime.datetime.fromtimestamp(current_time)
#team_plan[team_plan.time_start < ]
day_start = datetime.datetime(d.year, d.month, d.day, 18, 0, 0)
day_start

datetime.datetime(2016, 7, 22, 18, 0)

In [582]:
day_next = day_start + datetime.timedelta(1)

In [583]:
team_plan['dt_start'] = team_plan.time_start.apply(datetime.datetime.fromtimestamp)

In [599]:
st_name = 'ИРКУТСК-СОРТИРОВОЧНЫЙ'
team_plan[(team_plan.dt_start >= day_start) 
          & (team_plan.dt_start < day_next)
          & (team_plan.st_from_name == st_name) & (team_plan.state.isin([0, 1]))
         & (team_plan.depot_name == st_name)
         & (team_plan.st_to_name == 'БАТАРЕЙНАЯ')][['team', 'st_from_name', 'st_to_name', 'dt_start', 'loco', 'state']].sort_values('dt_start')

,team,st_from_name,st_to_name,dt_start,loco,state
39621,777700013028,ИРКУТСК-СОРТИРОВОЧНЫЙ,БАТАРЕЙНАЯ,2016-07-22 18:20:00,200200091211,1
30736,777700013168,ИРКУТСК-СОРТИРОВОЧНЫЙ,БАТАРЕЙНАЯ,2016-07-22 18:20:00,200200098197,1
39596,777700013026,ИРКУТСК-СОРТИРОВОЧНЫЙ,БАТАРЕЙНАЯ,2016-07-22 18:28:38,200200083020,1
30958,777700013167,ИРКУТСК-СОРТИРОВОЧНЫЙ,БАТАРЕЙНАЯ,2016-07-22 19:28:07,200200081936,1
8817,200200110159,ИРКУТСК-СОРТИРОВОЧНЫЙ,БАТАРЕЙНАЯ,2016-07-22 20:10:00,200200050952,1
2210,200200223785,ИРКУТСК-СОРТИРОВОЧНЫЙ,БАТАРЕЙНАЯ,2016-07-22 20:35:00,200200048996,1
3416,200200120486,ИРКУТСК-СОРТИРОВОЧНЫЙ,БАТАРЕЙНАЯ,2016-07-22 21:00:27,200200093794,1
18531,200200132310,ИРКУТСК-СОРТИРОВОЧНЫЙ,БАТАРЕЙНАЯ,2016-07-22 22:20:00,200200088747,1
7711,200200129611,ИРКУТСК-СОРТИРОВОЧНЫЙ,БАТАРЕЙНАЯ,2016-07-22 23:36:08,200200063979,1
8078,200200165890,ИРКУТСК-СОРТИРОВОЧНЫЙ,БАТАРЕЙНАЯ,2016-07-23 00:53:17,200200054913,1
